# Estimating Labor Market Returns to Education

**Note:** Most students taking this class are Duke MIDS students who have worked with `pandas` previously. As a result, these exercises are very light on basic pandas Series and DataFrame manipulations. If you are new to `pandas`, I would advise looking into some addition practice opportunities with `pandas`, as discussed on the [Advice for Non-MIDS Students](../not_a_mids_student.ipynb) page, such as DataCamp's "pandas Fundamentals" course. 

In this exercise, we're going to use data from the [American Communities Survey (ACS)](https://usa.ipums.org/usa/acs.shtml) to study the relationship betwen educational attainment and wages. The ACS is a survey conducted by the United States Census Bureau (though it is not "The Census," which is a counting of every person in the United States that takes place every 10 years) to measure numerous features of the US population. The data we will be working with includes about 100 variables from the 2017 ACS survey, and is a 10% sample of the ACS (which itself is a 1% sample of the US population, so we're working with about a 0.1% sample of the United States). 

This data comes from [IPUMS](https://usa.ipums.org/usa/), which provides a very useful tool for getting subsets of major survey datasets, not just from the US, but [from government statistical agencies the world over](https://international.ipums.org/international-action/sample_details).

This is *real* data, meaning that you are being provided the data as it is provided by IPUMS. Documentation for all variables used in this data can be found [here](https://usa.ipums.org/usa-action/variables/group) (you can either search by variable name to figure out the meaning of a variable in this data, or search for something you want to see if a variable with the right name is in this data). 

Within this data is information on both the educational background and current earnings of a representative sample of Americans. We will now use this data to estimate the labor-market returns to graduating high school and college, and to learn something about the meaning of an educational degree. 

### Exercise 1

Data for these [exercises can be found here](https://github.com/nickeubank/MIDS_Data/tree/master/US_AmericanCommunitySurvey). First, download `US_ACS_2017_10pct_sample.dta`. 

### Exercise 2

Now import `US_ACS_2017_10pct_sample.dta` into a pandas DataFrame. This can be done with the command `pd.read_stata`, which will read in files created in the program Stata (and which uses the file suffix `.dta`). This is a format commonly used by social scientists.

In [1]:
%load_ext lab_black
import pandas as pd
import numpy as np

# Download the data
acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

## Getting to Know Your Data

When you get a new dataset like this, it's good to start by trying to get a feel for its contents and organization. Toy datasets you sometimes get in classes are often very small, and easy to look at, but this is a pretty large dataset, so you can't just open it up and get a good sense of it. Here are some ways to get to know your data. 

**3)** How many rows are in your data?

In [2]:
len(acs)
# US population is a little over 300 million,
# so 300,000 observations is about right.

319004

**4)** How many columns are in your data?

In [3]:
len(acs.columns)

104

**5)** Let's see what variables are in this dataset. First, try to see them all using the command:

```
acs.columns
```


In [4]:
acs.columns

Index(['year', 'datanum', 'serial', 'cbserial', 'numprec', 'subsamp', 'hhwt',
       'hhtype', 'cluster', 'adjust',
       ...
       'migcounty1', 'migmet131', 'vetdisab', 'diffrem', 'diffphys', 'diffmob',
       'diffcare', 'diffsens', 'diffeye', 'diffhear'],
      dtype='object', length=104)

As you will see, `python` doesn't like to print out all the different variables. To get everything printed out, we can loop over all the columns and print them one at a time with the command:

```
for c in acs.columns: print(c)
```

Try it. 

In [5]:
for c in acs.columns:
    print(c)

year
datanum
serial
cbserial
numprec
subsamp
hhwt
hhtype
cluster
adjust
cpi99
region
stateicp
statefip
countyicp
countyfip
metro
city
citypop
strata
gq
farm
ownershp
ownershpd
mortgage
mortgag2
mortamt1
mortamt2
respmode
pernum
cbpernum
perwt
slwt
famunit
sex
age
marst
birthyr
race
raced
hispan
hispand
bpl
bpld
citizen
yrnatur
yrimmig
language
languaged
speakeng
hcovany
hcovpriv
hinsemp
hinspur
hinstri
hcovpub
hinscaid
hinscare
hinsva
hinsihs
school
educ
educd
gradeatt
gradeattd
schltype
degfield
degfieldd
degfield2
degfield2d
empstat
empstatd
labforce
occ
ind
classwkr
classwkrd
looking
availble
inctot
ftotinc
incwage
incbus00
incss
incwelfr
incinvst
incretir
incsupp
incother
incearn
poverty
migrate1
migrate1d
migplac1
migcounty1
migmet131
vetdisab
diffrem
diffphys
diffmob
diffcare
diffsens
diffeye
diffhear


**4)** That's a *lot* of variables, and definitely more than we need. In general, life is easier when working with these kinds of huge datasets if you can narrow down the number of variables a little. In this exercise, we will be looking at the relationship between education and wages, we need variables for: 

- Age
- Income
- Education
- Employment status (is the person actually working)

These quantities of interest correspond to the following variables in our data: `age`, `inctot`, `educ`, and `empstat`. 

Subset your data to just those variables. 

In [6]:
acs = acs[["age", "empstat", "inctot", "educ"]]

**5)** Now that we have a more manageable number of variables, it's often very useful to look at a handful of rows of your data. The easiest way to do this is probably the `.head()` method (which will show you the first five rows), or the `tail()` method, which will show you the last five rows. 

But to get a good sense of your data, it's often better to use the `sample()` command, which returns a random set of rows. As the first and last rows are sometimes not representative, a random set of rows can be very helpful. Try looking at a random sample of 20 rows (note: you don't have to run `.sample()` ten times to get ten rows. Look at the `.sample` help file if you're stuck. 

In [7]:
acs.sample(20)

age             empstat   inctot                       educ
299688  15                 n/a        0        grade 5, 6, 7, or 8
60572   18            employed     6000                   grade 12
30162   42  not in labor force        0          1 year of college
203613  46            employed    68000        5+ years of college
169945  68            employed    46400                   grade 12
267317  84  not in labor force    49000                   grade 12
267598  62  not in labor force    19900                   grade 12
225616  71  not in labor force     8900                   grade 12
95262    1                 n/a  9999999        n/a or no schooling
5717    71  not in labor force    94000         4 years of college
61423   53            employed    55000                   grade 12
189520  36            employed    40000         4 years of college
94082   82  not in labor force    22700                   grade 12
141775  16  not in labor force      500                   grade 10
244018  10                 n/a  9999999  nursery school to grade 4
18306   17  not in labor force        0                   grade 11
216971  27            employed    46300          1 year of college
285635  57            employed   110000         4 years of college
55683   30            employed    34000         2 years of college
70139   34            employed    30000        n/a or no schooling

**6)** Do you see any immediate problems? Write them down with your partner. 

In [8]:
# Uh, yup! People have incomes of 9 million?! And those people tend to be children?

**7)** One problem is that many people seem to have incomes of $9,999,999. Moreover, people with those incomes seem to be very young children. 

What you are seeing is one method (a relatively old one) for representing missing data. In this case, the value 9999999 is used to denote observations for which there is no data (Or more specifically, in this case observations where the person is too young to work, so their income value is missing). 

So let's begin by dropping anyone who has `inctot` equal to 9999999. 

In [9]:
acs = acs.loc[acs["inctot"] != 9999999]

**8)** OK, the other potential problem is that our data includes lots of people who are unemployed and people who are not in the labor force (this means they not only don't have a job, but also aren't looking for a job). For this analysis, we want to focus on the wages of people who are currently employed. So subset the dataset for the people for whom `empstat` is equal to "employed". 

Note that our decision to only look at people who are employed impacts how we should interpret the relationship we estimate between education and income. Because we are only looking at employed people, we will be estimating the relationship between education and income *for people who are employed*. That means that if education affects the *likelihood* someone is employed, we won't capture that in this analysis. (Economists all this the "intensive margin", while looking at whether people get jobs in the first place is called the "extensive margin".)

In [10]:
acs = acs.loc[
    acs.empstat == "employed",
]
acs.sample(10)

age   empstat  inctot                 educ
187997  45  employed  139000   4 years of college
306335  34  employed   60000   2 years of college
119915  21  employed    6050             grade 12
43453   31  employed   50000  5+ years of college
64370   37  employed   80000   4 years of college
159325  42  employed  100000  5+ years of college
312097  65  employed   18000             grade 12
121154  55  employed  120000  5+ years of college
199340  28  employed   22000             grade 12
104422  56  employed   32800             grade 12

**9)** Now let's turn to education. The `educ` variable seems to have a lot of discrete values. Let's see what values exist, and their distribution, using the `value_counts()` method. This is an *extremely* useful tool you'll use a lot! Try the following code (modified for the name of your dataset, of course):

```
acs['educ'].value_counts()
```

In [11]:
acs["educ"].value_counts()

grade 12                     47815
4 years of college           33174
1 year of college            22899
5+ years of college          20995
2 years of college           14077
grade 11                      2747
grade 5, 6, 7, or 8           2092
grade 10                      1910
n/a or no schooling           1291
grade 9                       1290
nursery school to grade 4      468
Name: educ, dtype: int64

In [12]:
a = acs["educ"]
a = [1, 3, 2]
pd.read_fwf?

Signature:
pd.read_fwf(
    filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]],
    colspecs='infer',
    widths=None,
    infer_nrows=100,
    **kwds,
)
Docstring:
Read a table of fixed-width formatted lines into DataFrame.

Also supports optionally iterating or breaking of the file
into chunks.

Additional help can be found in the `online docs for IO Tools
<https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html>`_.

Parameters
----------
filepath_or_buffer : str, path object or file-like object
    Any valid string path is acceptable. The string could be a URL. Valid
    URL schemes include http, ftp, s3, and file. For file URLs, a host is
    expected. A local file could be:
    ``file://localhost/path/to/table.csv``.

    If you want to pass in a path object, pandas accepts any
    ``os.PathLike``.

    By file-like object, we refer to objects with a ``read()`` method,
    such as a file handler (e.g. via builtin ``open`` function)
    or ``StringIO``.
colspecs : li

**10)** There are a lot of values in here, so let's just check a couple. What is the average value of `inctot` for people whose highest grade level is "grade 12" (in the US, that is someone who has graduated high school)?

In [13]:
acs.loc[acs['educ']=='grade 12', 'inctot'].mean?

Object `mean` not found.


**11)** What is the average income of someone who graduated college ("4 years of college")? What does that suggest is the value of getting a college degree after graduating high school?

In [14]:
acs.loc[acs["educ"] == "4 years of college", "inctot"].mean()

75485.05293301983

In [15]:
# That's a LOT better than the average high school income.

**12)** What is the average income for someone who has not finished high school? What does that suggest is the value of a high school diploma?

In [16]:
# Hasn't finished:
acs.loc[
    (acs["educ"] == "n/a or no schooling")
    | (acs["educ"] == "nursery school to grade 4")
    | (acs["educ"] == "grade 5, 6, 7, or 8")
    | (acs["educ"] == "grade 9")
    | (acs["educ"] == "grade 10")
    | (acs["educ"] == "grade 11"),
    "inctot",
].mean()

26226.45692998571

In [17]:
# Or the other way around:

acs.loc[
    ~(
        (acs["educ"] == "grade 12")
        | (acs["educ"] == "1 year of college")
        | (acs["educ"] == "2 years of college")
        | (acs["educ"] == "4 years of college")
        | (acs["educ"] == "5+ years of college")
    ),
    "inctot",
].mean()

26226.45692998571

In [18]:
# Or if you want to use a little trick:
less_than_college = [
    "n/a or no schooling",
    "nursery school to grade 4",
    "grade 5, 6, 7, or 8",
    "grade 9",
    "grade 10",
    "grade 11",
]

acs.loc[acs["educ"].isin(less_than_college), "inctot"].mean()

26226.45692998571

In [19]:
# High school diploma is also worth a lot -- they earn almost 50% more than
# those without a high school degree

(38957.76 - 26226.45) / 26226.45

0.48543779276265

**13)** Complete the following table:

- Average income for someone who has not finished high school: _________
- Average income for someone who only completed 9th grade: _________
- Average income for someone who only completed 10th grade: _________
- Average income for someone who only completed 11th grade: _________
- Average income for someone who finished high school (12th grade) but never started college: _________
- Average income for someone who completed 4 year of college (in the US, this means graduating college): _________

In [20]:
for level in ["grade 9", "grade 10", "grade 11", "grade 12", "4 years of college"]:
    avg_income = acs.loc[acs["educ"] == level, "inctot"].mean()
    print("those who have finished {} earn {:,.2f}".format(level, avg_income))

those who have finished grade 9 earn 27,171.91
those who have finished grade 10 earn 23,018.80
those who have finished grade 11 earn 21,541.69
those who have finished grade 12 earn 38,957.76
those who have finished 4 years of college earn 75,485.05


**14)** Why do you think there is no benefit from moving from grade 9 to grade 10, or grade 10 to grade 11, but there is a huge benefit to moving from grade 11 to graduating high school (grade 12)?

In [21]:
# I mean... the answer is below.

## Take-aways

Congratulations! You just discovered "the sheepskin effect!": people with degrees tend to earn substantially more than people who have *almost* as much education, but don't have an actual degree. 

In economics, this is viewed as evidence that the reason employers pay people with high school degrees more than those without degree is *not* that they think those who graduated high school have learned specific, useful skills. If that were the case, we would expect employee earnings to rise with every year of high school, since in each year of high school we learn more. 

Instead, this suggests employees pay high school graduates more because they think *the kind of people* who can finish high school are the *kind of people* who are likely to succeed at their jobs. Finishing high school, in other words, isn't about accumulating specific knowledge; it's about showing that you *are the kind of person* who can rise to the challenge of finishing high school, also suggesting you are also the kind of person who can succeed as an employee. 

(Obviously, this does not tell us whether that is an *accurate* inference, just that that seems to be how employeers think.) 

In other words, in the eyes of employers, a high school degree is a *signal* about the kind of person you are, not certification that you've learned a specific set of skills (an idea that earned [Michael Spence](https://en.wikipedia.org/wiki/Michael_Spence) a Nobel Prize in Economics). 

## Want More Practice?

**(14)** The US Census isn't the only organization that published surveys of their populations -- most countries in the world conduct regular surveys, and most publish versions of their data online. 

In addition to making US Census data available to researchers, IPUMS (through "IPUMS International") also makes survey data available from a wide range of countries. 

So if you'd like more practice with basic data manipulations, go to [IPUMS International](https://international.ipums.org/international-action/variables/group) and download information on income and education from the country of your choice:

1. Click "Select Samples" to pick the country and year from which you would like data, then click "Submit Sample Selections". *Note that most countries don't include income as a variable! Asking income is very sensitive, so is often left off surveys*. You can get income directly from surveys of Mexico, Israel, Canada, India (in 2004), Italy, Panama, and Venezuela. But you *can* get "income proxies" (variables that are very correlated with income) like household construction materials (people with unfinished dirt floors tend to be poorer than people with concrete floors) for other countries like most African and Latin American surveys. Also: note there are non-IPUMS surveys which often *do* provide this data, they just aren't here. 
2. Select variables (most likely under the "Person" tab) for education and income, or under "Dwelling Characteristics" if you're looking for wealth proxies. 
3. Click "View Cart"
4. Click "Select Data Extract"
5. Change the data format to .dta or to .csv.
6. Click "Submit Extract"
7. Login with the IPUMS account you created during week 1 (if you didn't do that (as assigned!!), register for IPUMS now). 
8. When extract is ready, download!

**(15)** Repeat these exercises for your new dataset!